In [3]:
1+1

2

In [4]:
import pyspark
from pyspark.sql import SparkSession
import os

In [5]:
pyspark.__version__

'2.4.1'

In [6]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/lib/pravega-connectors-spark-0.4.0-SNAPSHOT.jar,/home/jovyan/lib/pravega-keycloak-credentials-0.4.0-2030.d99411b-0.0.1-020.26736d2-shadow.jar pyspark-shell'
#os.environ['pravega_client_auth_method'] = 'Bearer'
#os.environ['pravega_client_auth_loadDynamic'] = 'true'
#os.environ['KEYCLOAK_SERVICE_ACCOUNT_FILE'] = '/home/jovyan/keycloak.json'

In [7]:
pravega_jars = [
    'pravega-connectors-spark-0.4.0-SNAPSHOT.jar', 
    'pravega-keycloak-credentials-0.4.0-2030.d99411b-0.0.1-020.26736d2-shadow.jar']
spark_jars_conf = ','.join([os.path.join('/usr/local/share/java', j) for j in pravega_jars])
spark = (SparkSession
             .builder
             .appName('test1')
             .config('spark.jars', spark_jars_conf)
             .getOrCreate()
             )
spark.conf.set('spark.sql.shuffle.partitions', '1')
spark.conf.set('spark.sql.execution.arrow.enabled', 'true')

In [10]:
spark.sparkContext.getConf().getAll()

[('spark.jars',
  '/usr/local/share/java/pravega-connectors-spark-0.4.0-SNAPSHOT.jar,/usr/local/share/java/pravega-keycloak-credentials-0.4.0-2030.d99411b-0.0.1-020.26736d2-shadow.jar'),
 ('spark.repl.local.jars',
  'file:///usr/local/share/java/pravega-connectors-spark-0.4.0-SNAPSHOT.jar,file:///usr/local/share/java/pravega-keycloak-credentials-0.4.0-2030.d99411b-0.0.1-020.26736d2-shadow.jar'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'jupyter-claudio'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '37837'),
 ('spark.app.id', 'local-1556563928893'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'test1')]

In [11]:
spark.createDataFrame([1, 2, 3, 4], 'int').show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
+-----+



In [12]:
controller = 'tcp://nautilus-pravega-controller.nautilus-pravega.svc.cluster.local:9090'
scope = 'examples'
df = (spark
      .read
      .format("pravega")
      .option("controller", controller)
      .option("scope", scope)
      .option("stream", "sensors")
      .load()
      )
df.show()

+--------------------+--------+-------+----------+------+
|               event|   scope| stream|segment_id|offset|
+--------------------+--------+-------+----------+------+
|[7B 22 74 69 6D 6...|examples|sensors|         0|     0|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   127|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   254|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   381|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   509|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   636|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   762|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   890|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1018|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1146|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1273|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1401|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1529|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1656|
|[7B 22 74 69 

In [13]:
df = df.limit(10).cache()

In [14]:
df.show()

+--------------------+--------+-------+----------+------+
|               event|   scope| stream|segment_id|offset|
+--------------------+--------+-------+----------+------+
|[7B 22 74 69 6D 6...|examples|sensors|         0|     0|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   127|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   254|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   381|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   509|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   636|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   762|
|[7B 22 74 69 6D 6...|examples|sensors|         0|   890|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1018|
|[7B 22 74 69 6D 6...|examples|sensors|         0|  1146|
+--------------------+--------+-------+----------+------+



In [15]:
def f(row):
    filename = '/tmp/file-%d' % (row.offset,)
    with open(filename, 'wb') as output:
        output.write(row.event)
df.foreach(f)